**Troubleshooting Scripts - Statistics**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Analyzing Statistics - DBCC SHOW\_STATISTICS**

In [ ]:
DBCC SHOW_STATISTICS('Schema.Table','Index');

**Analyzing Statistics - sys.dm\_db\_stats\_histogram** (SQL Server 2016 SP1 CU2+)

Specify table and index names in WHERE clause

In [ ]:
SELECT 
	sp.*
FROM 
	sys.tables t WITH (NOLOCK) JOIN sys.indexes i WITH (NOLOCK) ON
		t.object_id = i.object_id
	CROSS APPLY
		sys.dm_db_stats_histogram(t.object_id, i.index_id) sp
WHERE
	t.object_id = OBJECT_ID(N'Schema.Table') AND
	i.name = 'Index'
OPTION (MAXDOP 1, RECOMPILE);

**Analyzing Statistics Properties - sys.dm\_db\_stats\_properties** 

Filter by table name if needed in WHERE clause

In [ ]:
SELECT
    s.stats_id AS [Stat ID]
    ,sc.name + '.' + t.name AS [Table]
    ,s.name AS [Statistics]
    ,p.last_updated
    ,p.rows
    ,p.rows_sampled
    ,p.modification_counter AS [Mod Count]
FROM
    sys.stats s WITH (NOLOCK) JOIN sys.tables t WITH (NOLOCK) ON
        s.object_id = t.object_id
    JOIN sys.schemas sc WITH (NOLOCK) ON
        t.schema_id = sc.schema_id
    OUTER APPLY
        sys.dm_db_stats_properties(t.object_id,s.stats_id) p
--WHERE
--	s.object_id = OBJECT_ID(N'<Table Name>')
ORDER BY
    p.last_updated
OPTION (RECOMPILE, MAXDOP 1);